In [13]:
# Cell 1: Import necessary libraries
import matplotlib.pyplot as plt  # Used for plotting graphs
import numpy as np  # Provides support for large, multi-dimensional arrays and matrices
import cv2  # OpenCV library for handling image operations
from tqdm import trange  # Provides a progress bar for loops
from gymnasium.utils.env_checker import check_env  # Utility for checking and validating environments
from flygym import Fly, Camera  # Importing the Fly and Camera classes for simulation
from flygym.arena import FlatTerrain, OdorArena  # Importing terrain types for the simulation environment
from flygym.examples.turning_controller import HybridTurningNMF  # A neural model for turning and navigation
from pathlib import Path  # Library for easier file path management

# Cell 2: Define parameters
output_directory = Path("./outputs/outputs_finaux")
output_directory.mkdir(exist_ok=True)  # Ensure the directory exists where simulation outputs will be stored

# Cell 4: Create the arena
peak_odor_intensity = np.array([[1, 0], [0, 0], [0, 0]])  # Highest intensity of odor detectable at sources
odor_source_A = np.array([[-2, 1, 0.2], [-5, -5, 0.2], [-1, 5, 0.2]])  # Coordinates of dynamic odor sources
marker_colors = [[255, 127, 14], [0, 0, 128], [0, 0, 128]]  # Define marker colors (orange and blue)
marker_colors = np.array([np.array(color) / 255 for color in marker_colors])  # Normalize colors to range [0, 1]

# Configure the simulation arena with a flat terrain and odor sources
arena = OdorArena(
    odor_source=odor_source_A,
    peak_odor_intensity=peak_odor_intensity,
    diffuse_func=lambda x: x ** -2,  # Diffusion function defining how odor disperses
    marker_colors=marker_colors,
    marker_size=0.3  # Size of markers in the simulation environment
)

# Cell 5: Setup the fly and camera
# Define sensor placements on the fly's body for environmental interaction
contact_sensor_placements = [
    f"{leg}{segment}" for leg in ["LF", "LM", "LH", "RF", "RM", "RH"]
    for segment in ["Tibia", "Tarsus1", "Tarsus2", "Tarsus3", "Tarsus4", "Tarsus5"]
]

# Initialize the Fly object with specified position and sensors
fly = Fly(
    spawn_pos=(-1, 1, 0.2),  # Position the fly behind the origin point
    spawn_orientation=(0, 0, np.pi / 2),  # Orientation facing left
    contact_sensor_placements=contact_sensor_placements,
    enable_olfaction=True,  # Enable the fly's olfaction sensors
    enable_adhesion=True,  # Allow the fly to adhere to surfaces
    draw_adhesion=False  # Disable drawing of adhesion for clarity
)

# Configure the camera to observe the simulation
cam = Camera(
    fly=fly,
    camera_id="birdeye_cam",
    play_speed=0.2,  # Speed at which the simulation plays
    window_size=(1000, 608)  # Resolution of the camera view
)

# Initialize the simulation with the fly and camera settings
sim = HybridTurningNMF(
    fly=fly,
    cameras=[cam],
    arena=arena
)

# Cell 3: Define utility functions
def draw_white_area_corners(frame, bounds, color=(0, 0, 255), size=5):
    """Draw rectangles at the corners of a defined area to indicate boundaries."""
    x1, y1 = bounds[0]
    x2, y2 = bounds[1]
    corners = [(x1, y1), (x1, y2), (x2, y1), (x2, y2)]
    for corner in corners:
        pixel_x = int((corner[0] + 6.75) * 100)  # Scaling for visualization
        pixel_y = int((corner[1] - 0.46) * 100)
        cv2.circle(frame, (pixel_x, pixel_y), size, color, -1)
    return frame

def is_in_white_area(position, bounds):
    """Check if a position is within a specified bounding area."""
    x, y = position[:2]
    x1, y1 = bounds[0]
    x2, y2 = bounds[1]
    return x1 <= x <= x2 and y1 <= y <= y2

# Cell 7: Function to run the simulation
def run_simulation(fly_mobile=True, source_mobile=False, bounds=False, white_area_bounds=white_area_bounds, bounds_speed=np.array([[0.03, 0.0], [0.03, 0.0]]), n_delays=n_delays):
    """ Conducts the simulation with configured parameters. """
    global curr_time, initial_xpos, x_positions, times, obs_hist, odor_history, antennae_positions, O, HRC_model
    x_positions = []
    times = []
    obs_hist = []
    odor_history = []
    antennae_positions = []
    delayed_intensities = []
    obs, _ = sim.reset()  # Reset the simulation to start fresh.

    for i in trange(num_decision_steps):
        antennae_pos = sim.physics.bind(fly._antennae_sensors).sensordata
        antennae_pos = antennae_pos.reshape(-1, 3)  # Reformat sensor data for processing.
        antennae_positions.append(antennae_pos)
        if bounds:  # Adjust odor intensity based on position within bounds.
            white_area_bounds += bounds_speed
            odor_intensity = arena.get_olfaction(antennae_pos)
            for idx, sensor_pos in enumerate(antennae_pos):
                if not is_in_white_area(sensor_pos, white_area_bounds):
                    odor_intensity[:, idx] = 0
                    obs["odor_intensity"] = odor_intensity.astype(np.float32)

        attractive_intensities = np.average(
            obs["odor_intensity"][0, :].reshape(2, 2), axis=0, weights=[9, 1]
        )
        aversive_intensities = np.average(
            obs["odor_intensity"][1, :].reshape(2, 2), axis=0, weights=[10, 0]
        )

        control_signal = np.zeros((2,))

        delayed_intensities = O.pop(0)
        current_intensities = [obs["odor_intensity"][0, :][1], obs["odor_intensity"][0, :][3]]
        O.append(current_intensities)
        HRC_model.append(delayed_intensities[0] * current_intensities[1] - delayed_intensities[1] * current_intensities[0])

        odor_history.append(attractive_intensities + aversive_intensities)
        if source_mobile:
            xpos += 0.03
        if bounds:
            xposA += 0.03
            xposB += 0.03

        for j in range(physics_steps_per_decision_step):
            curr_time += sim.timestep
            times.append(curr_time)
            x_positions.append(xpos)

            if source_mobile:
                sim.physics.bind(marker).mocap_pos[:2] = [xpos, 1]
                arena.odor_source[0, :2] = [xpos, 1]
            if bounds:
                sim.physics.bind(markerA).mocap_pos[:2] = [xposA, -5]
                arena.odor_source[1, :2] = [xposA, -5]
                sim.physics.bind(markerB).mocap_pos[:2] = [xposB, 5]
                arena.odor_source[2, :2] = [xposB, 5]

            obs, _, _, _, _ = sim.step(control_signal)
            rendered_img = sim.render()[0]
            if rendered_img is not None:
                hrc_value = HRC_model[-1] if HRC_model else 0
                im = cv2.putText(
                    rendered_img,
                    f"HRC Model: {hrc_value:.8f}",
                    (100, 50),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 0, 0),
                    2,
                    cv2.LINE_AA
                )
                cam._frames[-1] = im

            obs_hist.append(obs)



ValueError: Compile error raised by Mujoco; run again with --pymjcf_debug for additional debug information.
XML Error: attribute 'rgba' does not have enough data
Element 'geom', line 125

<geom name="groundblock_0" class="/" type="capsule" size="0.29999999999999999 0.29999999999999999" rgba="1 0.49803921568627452 0.054901960784313725"/>

In [9]:
run_simulation(fly_mobile=False, source_mobile=False,bounds=True)

100%|██████████| 200/200 [13:29<00:00,  4.05s/it]


In [10]:
x = list(range(num_decision_steps))
print(len(x))
#divide x by 100
x = [((i+1)*48) / 200 for i in x]
x

200


[0.24,
 0.48,
 0.72,
 0.96,
 1.2,
 1.44,
 1.68,
 1.92,
 2.16,
 2.4,
 2.64,
 2.88,
 3.12,
 3.36,
 3.6,
 3.84,
 4.08,
 4.32,
 4.56,
 4.8,
 5.04,
 5.28,
 5.52,
 5.76,
 6.0,
 6.24,
 6.48,
 6.72,
 6.96,
 7.2,
 7.44,
 7.68,
 7.92,
 8.16,
 8.4,
 8.64,
 8.88,
 9.12,
 9.36,
 9.6,
 9.84,
 10.08,
 10.32,
 10.56,
 10.8,
 11.04,
 11.28,
 11.52,
 11.76,
 12.0,
 12.24,
 12.48,
 12.72,
 12.96,
 13.2,
 13.44,
 13.68,
 13.92,
 14.16,
 14.4,
 14.64,
 14.88,
 15.12,
 15.36,
 15.6,
 15.84,
 16.08,
 16.32,
 16.56,
 16.8,
 17.04,
 17.28,
 17.52,
 17.76,
 18.0,
 18.24,
 18.48,
 18.72,
 18.96,
 19.2,
 19.44,
 19.68,
 19.92,
 20.16,
 20.4,
 20.64,
 20.88,
 21.12,
 21.36,
 21.6,
 21.84,
 22.08,
 22.32,
 22.56,
 22.8,
 23.04,
 23.28,
 23.52,
 23.76,
 24.0,
 24.24,
 24.48,
 24.72,
 24.96,
 25.2,
 25.44,
 25.68,
 25.92,
 26.16,
 26.4,
 26.64,
 26.88,
 27.12,
 27.36,
 27.6,
 27.84,
 28.08,
 28.32,
 28.56,
 28.8,
 29.04,
 29.28,
 29.52,
 29.76,
 30.0,
 30.24,
 30.48,
 30.72,
 30.96,
 31.2,
 31.44,
 31.68,
 31.92,
 32